In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
from scipy import stats

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✅ Bibliotecas cargadas")

## 1️⃣ Empalme de Datos

### ¿Por qué empalmar datos?

En campo, se realizan múltiples mediciones con diferentes configuraciones de **MN/2**. El empalme combina estas mediciones para obtener una curva única y representativa.

**Problema común**: Múltiples mediciones en el mismo **AB/2** con diferentes **MN/2**

In [ ]:
# Simular datos con múltiples mediciones en mismo AB/2
datos_repetidos = pd.DataFrame({
    'AB/2': [10, 10, 10, 20, 20, 20, 30, 30, 40, 40, 60, 60, 90, 90],
    'MN/2': [1, 1, 3, 1, 3, 3, 3, 9, 9, 9, 9, 27, 27, 27],
    'pa (Ω*m)': [85, 88, 92, 110, 115, 108, 125, 130, 135, 132, 128, 125, 120, 118]
})

print("📋 Datos con múltiples mediciones por AB/2:")
display(datos_repetidos)

print(f"\n📊 Total de mediciones: {len(datos_repetidos)}")
print(f"🔢 AB/2 únicos: {datos_repetidos['AB/2'].nunique()}")

In [ ]:
# Realizar empalme (promedio por AB/2)
datos_empalmados = datos_repetidos.groupby('AB/2').agg({
    'pa (Ω*m)': 'mean',
    'MN/2': 'mean'
}).reset_index()

print("✅ Datos empalmados (promediados por AB/2):")
display(datos_empalmados)

print(f"\n📉 Reducción: {len(datos_repetidos)} → {len(datos_empalmados)} puntos")

In [ ]:
# Comparación visual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Datos originales
ax1.loglog(datos_repetidos['AB/2'], datos_repetidos['pa (Ω*m)'], 
           'o', markersize=8, alpha=0.6, label='Mediciones múltiples')
ax1.set_xlabel('AB/2 (m)', fontweight='bold')
ax1.set_ylabel('Resistividad aparente (Ω·m)', fontweight='bold')
ax1.set_title('Datos Originales (con repeticiones)', fontweight='bold')
ax1.grid(True, which='both', alpha=0.3)
ax1.legend()

# Datos empalmados
ax2.loglog(datos_empalmados['AB/2'], datos_empalmados['pa (Ω*m)'], 
           's-', markersize=10, linewidth=2, color='green', label='Datos empalmados')
ax2.set_xlabel('AB/2 (m)', fontweight='bold')
ax2.set_ylabel('Resistividad aparente (Ω·m)', fontweight='bold')
ax2.set_title('Datos Empalmados (promediados)', fontweight='bold')
ax2.grid(True, which='both', alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print("✅ El empalme reduce el ruido y simplifica la curva")

## 2️⃣ Suavizado de Datos

### Técnicas de suavizado:

1. **Media móvil**: Promedio de ventana deslizante
2. **Savitzky-Golay**: Ajuste polinomial local
3. **Media exponencial**: Peso decreciente hacia atrás

In [ ]:
# Generar datos con ruido
np.random.seed(42)
ab2_values = np.logspace(0, 2.3, 25)
resistividad_teorica = 50 + 60 * np.log10(ab2_values) - 10 * np.sin(np.log10(ab2_values) * 3)
ruido = np.random.normal(0, 8, len(ab2_values))
resistividad_con_ruido = resistividad_teorica + ruido

datos_ruido = pd.DataFrame({
    'AB/2': ab2_values,
    'pa (Ω*m)': resistividad_con_ruido
})

print(f"📊 Datos generados: {len(datos_ruido)} puntos con ruido gaussiano")

In [ ]:
# Aplicar diferentes técnicas de suavizado
ventana = 5

# 1. Media móvil
media_movil = np.convolve(datos_ruido['pa (Ω*m)'], np.ones(ventana)/ventana, mode='same')

# 2. Savitzky-Golay
savgol = savgol_filter(datos_ruido['pa (Ω*m)'], window_length=9, polyorder=2)

# 3. Media exponencial
media_exp = datos_ruido['pa (Ω*m)'].ewm(span=ventana, adjust=False).mean()

print("✅ Filtros aplicados:")
print(f"   1. Media móvil (ventana={ventana})")
print(f"   2. Savitzky-Golay (ventana=9, orden=2)")
print(f"   3. Media exponencial (span={ventana})")

In [ ]:
# Visualizar comparación de filtros
plt.figure(figsize=(14, 8))

plt.semilogx(datos_ruido['AB/2'], datos_ruido['pa (Ω*m)'], 
             'o', alpha=0.4, markersize=6, label='Datos con ruido', color='gray')
plt.semilogx(datos_ruido['AB/2'], media_movil, 
             '-', linewidth=2, label='Media móvil', color='blue')
plt.semilogx(datos_ruido['AB/2'], savgol, 
             '-', linewidth=2, label='Savitzky-Golay', color='green')
plt.semilogx(datos_ruido['AB/2'], media_exp, 
             '-', linewidth=2, label='Media exponencial', color='red')
plt.semilogx(datos_ruido['AB/2'], resistividad_teorica, 
             '--', linewidth=2, label='Teórico (sin ruido)', color='black')

plt.xlabel('AB/2 (m)', fontsize=12, fontweight='bold')
plt.ylabel('Resistividad aparente (Ω·m)', fontsize=12, fontweight='bold')
plt.title('Comparación de Técnicas de Suavizado', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Observaciones:")
print("   - Media móvil: Simple pero puede perder detalles")
print("   - Savitzky-Golay: Preserva mejor picos y valles")
print("   - Media exponencial: Da más peso a datos recientes")

## 3️⃣ Detección de Valores Atípicos (Outliers)

Los outliers pueden deberse a:
- Errores de medición
- Ruido eléctrico
- Contacto deficiente de electrodos

### Métodos de detección:
- **Z-score**: Distancia en desviaciones estándar
- **IQR (Rango intercuartílico)**: Basado en cuartiles

In [ ]:
# Agregar outliers artificiales
datos_con_outliers = datos_ruido.copy()
outlier_indices = [5, 15, 20]
datos_con_outliers.loc[outlier_indices, 'pa (Ω*m)'] *= 2.5

print("📊 Datos con outliers artificiales agregados")

In [ ]:
# Detección con Z-score
z_scores = np.abs(stats.zscore(datos_con_outliers['pa (Ω*m)']))
umbral_z = 2.5
outliers_z = z_scores > umbral_z

print(f"🔍 Detección con Z-score (umbral={umbral_z}):")
print(f"   Outliers detectados: {outliers_z.sum()}")

# Detección con IQR
Q1 = datos_con_outliers['pa (Ω*m)'].quantile(0.25)
Q3 = datos_con_outliers['pa (Ω*m)'].quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = (datos_con_outliers['pa (Ω*m)'] < (Q1 - 1.5 * IQR)) | \
               (datos_con_outliers['pa (Ω*m)'] > (Q3 + 1.5 * IQR))

print(f"\n🔍 Detección con IQR:")
print(f"   Q1: {Q1:.1f}, Q3: {Q3:.1f}, IQR: {IQR:.1f}")
print(f"   Outliers detectados: {outliers_iqr.sum()}")

In [ ]:
# Visualizar outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfica con outliers marcados
axes[0].semilogx(datos_con_outliers['AB/2'], datos_con_outliers['pa (Ω*m)'], 
                 'o-', markersize=6, label='Datos')
axes[0].semilogx(datos_con_outliers.loc[outliers_z, 'AB/2'], 
                 datos_con_outliers.loc[outliers_z, 'pa (Ω*m)'], 
                 'ro', markersize=12, label='Outliers (Z-score)', markeredgewidth=2)
axes[0].set_xlabel('AB/2 (m)', fontweight='bold')
axes[0].set_ylabel('Resistividad aparente (Ω·m)', fontweight='bold')
axes[0].set_title('Detección de Outliers', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(datos_con_outliers['pa (Ω*m)'], vert=True)
axes[1].set_ylabel('Resistividad aparente (Ω·m)', fontweight='bold')
axes[1].set_title('Diagrama de Caja', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Datos limpios (sin outliers)
datos_limpios = datos_con_outliers[~outliers_z].copy()
print(f"\n✅ Datos limpios: {len(datos_limpios)} puntos (eliminados {outliers_z.sum()})")

## 4️⃣ Pipeline Completo de Preprocesamiento

In [ ]:
def preprocesar_sev(datos, empalme=True, filtro='savgol', remover_outliers=True):
    """
    Pipeline completo de preprocesamiento de datos SEV.
    
    Parámetros:
    -----------
    datos : DataFrame con columnas 'AB/2' y 'pa (Ω*m)'
    empalme : bool, realizar empalme por AB/2
    filtro : str, tipo de filtro ('savgol', 'media_movil', 'exponencial')
    remover_outliers : bool, eliminar valores atípicos
    
    Retorna:
    --------
    DataFrame procesado
    """
    resultado = datos.copy()
    
    print("🔧 Iniciando preprocesamiento...")
    print(f"   Datos iniciales: {len(resultado)} puntos\n")
    
    # 1. Empalme
    if empalme and 'AB/2' in resultado.columns:
        n_antes = len(resultado)
        resultado = resultado.groupby('AB/2')['pa (Ω*m)'].mean().reset_index()
        print(f"✅ Empalme: {n_antes} → {len(resultado)} puntos")
    
    # 2. Remover outliers
    if remover_outliers:
        z_scores = np.abs(stats.zscore(resultado['pa (Ω*m)']))
        mask = z_scores < 2.5
        n_antes = len(resultado)
        resultado = resultado[mask].reset_index(drop=True)
        print(f"✅ Outliers removidos: {n_antes} → {len(resultado)} puntos")
    
    # 3. Suavizado
    if filtro == 'savgol' and len(resultado) >= 9:
        resultado['pa (Ω*m)'] = savgol_filter(resultado['pa (Ω*m)'], 9, 2)
        print(f"✅ Filtro aplicado: Savitzky-Golay")
    elif filtro == 'media_movil':
        ventana = min(5, len(resultado))
        resultado['pa (Ω*m)'] = np.convolve(resultado['pa (Ω*m)'], 
                                             np.ones(ventana)/ventana, mode='same')
        print(f"✅ Filtro aplicado: Media móvil (ventana={ventana})")
    elif filtro == 'exponencial':
        resultado['pa (Ω*m)'] = resultado['pa (Ω*m)'].ewm(span=5, adjust=False).mean()
        print(f"✅ Filtro aplicado: Media exponencial")
    
    print(f"\n✅ Preprocesamiento completado: {len(resultado)} puntos finales")
    return resultado

print("✅ Función de preprocesamiento definida")

In [ ]:
# Aplicar pipeline completo
datos_procesados = preprocesar_sev(
    datos_con_outliers, 
    empalme=False,
    filtro='savgol',
    remover_outliers=True
)

In [ ]:
# Comparación antes/después
plt.figure(figsize=(12, 6))

plt.semilogx(datos_con_outliers['AB/2'], datos_con_outliers['pa (Ω*m)'], 
             'o', alpha=0.4, markersize=6, label='Datos originales (con outliers)', color='gray')
plt.semilogx(datos_procesados['AB/2'], datos_procesados['pa (Ω*m)'], 
             's-', linewidth=2.5, markersize=8, label='Datos procesados', color='green')

plt.xlabel('AB/2 (m)', fontsize=12, fontweight='bold')
plt.ylabel('Resistividad aparente (Ω·m)', fontsize=12, fontweight='bold')
plt.title('Comparación: Datos Originales vs Procesados', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Los datos procesados están listos para inversión")

## 📝 Resumen

En este tutorial aprendiste:

✅ **Empalme de datos**: Combinar múltiples mediciones  
✅ **Suavizado**: Media móvil, Savitzky-Golay, media exponencial  
✅ **Detección de outliers**: Z-score e IQR  
✅ **Pipeline completo**: Función automatizada de preprocesamiento  

### 🚀 Próximo paso

En el **Tutorial 3** aprenderás sobre:
- Inversión de datos SEV con PyGIMLi
- Interpretación de modelos de capas
- Análisis de incertidumbre

---

**VESPY** - Vertical Electrical Sounding in Python  
📧 josemariagarciamarquez2.72@gmail.com  
🌐 [Web](https://josemariagarciamarquez.github.io/webjoma/) • ☕ [Patreon](https://www.patreon.com/chemitas)